<a href="https://colab.research.google.com/github/MarcosNMeireles/FullStack/blob/main/C%C3%B3pia_de_TensorFlow_with_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


## Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
import joblib
import os
import random
from google.colab import files
from google.colab import drive  # Import para Google Drive

# Função para fazer o upload do arquivo no Colab
def selecionar_arquivo():
    print("Selecione o arquivo de sorteios (em formato .xlsx):")
    arquivo = files.upload()
    caminho_arquivo = next(iter(arquivo))  # Pega o primeiro arquivo carregado
    return caminho_arquivo

# Função para montar o Google Drive
def montar_drive():
    drive.mount('/content/drive')
    caminho_modelo_drive = '/content/drive/MyDrive/modelo_sorteio.pkl'  # Caminho no Google Drive
    return caminho_modelo_drive

# Função para carregar o modelo do Google Drive
def carregar_modelo(caminho_modelo_drive):
    if os.path.exists(caminho_modelo_drive):
        print("Carregando modelo treinado do Google Drive...")
        return joblib.load(caminho_modelo_drive)
    return None

# Função para salvar o modelo no Google Drive
def salvar_modelo(modelo, caminho_modelo_drive):
    print("Salvando o modelo treinado no Google Drive...")
    joblib.dump(modelo, caminho_modelo_drive)

# Função para criar features adicionais
def criar_features_adicionais(dados):
    colunas_numeros = ['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']

    # Características estatísticas
    dados['media_numeros'] = dados[colunas_numeros].mean(axis=1)
    dados['std_numeros'] = dados[colunas_numeros].std(axis=1)
    dados['soma_numeros'] = dados[colunas_numeros].sum(axis=1)

    # Diferenças entre números consecutivos
    for i in range(len(colunas_numeros) - 1):
        dados[f'diff_{i+1}'] = dados[colunas_numeros[i+1]] - dados[colunas_numeros[i]]

    # Análise de paridade
    for col in colunas_numeros:
        dados[f'{col}_par'] = dados[col] % 2 == 0
    dados['qtd_pares'] = dados[[f'{col}_par' for col in colunas_numeros]].sum(axis=1)

    return dados

# Função para simulação de Monte Carlo
def simular_monte_carlo(dados, num_simulacoes=10000):
    resultados = dados[['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']].values[-2:]
    acertou = False
    contagem_simulacoes = 0

    while not acertou and contagem_simulacoes < num_simulacoes:
        contagem_simulacoes += 1
        simulacao = sorted(random.sample(range(1, 61), 6))
        if any(np.array_equal(simulacao, resultado) for resultado in resultados):
            acertou = True
            print(f"Simulação acertou após {contagem_simulacoes} tentativas: {simulacao}")
            return simulacao
    print("Simulação não acertou em 10.000 tentativas.")
    return None

# Função para treinar o modelo ou carregar do Google Drive
def treinar_modelo(dados, X_pca, y, caminho_modelo_drive):
    modelo = carregar_modelo(caminho_modelo_drive)
    if modelo is None:
        print("Treinando um novo modelo...")
        modelo_rf = RandomForestClassifier(n_estimators=200, max_depth=15, min_samples_split=5, min_samples_leaf=2, random_state=42)
        modelo_xgb = xgb.XGBClassifier(n_estimators=200, max_depth=8, learning_rate=0.01, subsample=0.8, colsample_bytree=0.8, eval_metric='logloss', random_state=42)
        modelo_gb = GradientBoostingClassifier(n_estimators=200, max_depth=8, learning_rate=0.01, subsample=0.8, random_state=42)

        modelo = MultiOutputClassifier(estimator=VotingClassifier(estimators=[('rf', modelo_rf), ('xgb', modelo_xgb), ('gb', modelo_gb)], voting='soft'))
        modelo.fit(X_pca, y)

        salvar_modelo(modelo, caminho_modelo_drive)
    return modelo

# Executando o código principal
caminho_arquivo = selecionar_arquivo()

if not caminho_arquivo:
    print("Nenhum arquivo foi selecionado.")
else:
    # Montar o Google Drive para salvar o modelo
    caminho_modelo_drive = montar_drive()

    # Carregar os dados do arquivo
    dados = pd.read_excel(caminho_arquivo, engine='openpyxl')
    print("Colunas carregadas:", dados.columns)

    if len(dados.columns) > 7:
        dados = dados.iloc[:, :7]

    dados.columns = ['Sorteio', 'Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']

    # Criar features avançadas
    dados = criar_features_adicionais(dados)
    colunas_target = ['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']
    colunas_features = [col for col in dados.columns if col not in colunas_target + ['Sorteio']]

    X = dados[colunas_features]
    y = dados[colunas_target]

    imputer = SimpleImputer(strategy='mean')
    X = imputer.fit_transform(X)

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    pca = PCA(n_components=0.98)
    X_pca = pca.fit_transform(X_scaled)

    # Treinando ou carregando o modelo
    modelo = treinar_modelo(dados, X_pca, y, caminho_modelo_drive)

    # Predições e análise
    predicoes = modelo.predict(X_pca)
    acuracias = [accuracy_score(y.iloc[:, i], predicoes[:, i]) for i in range(y.shape[1])]
    acuracia_media = np.mean(acuracias)
    print(f"Acurácia média nas predições: {acuracia_media * 100:.2f}%")

    # Simulação de Monte Carlo
    sequencia_simulada = simular_monte_carlo(dados)
    print("Sequência simulada:", sequencia_simulada)


Selecione o arquivo de sorteios (em formato .xlsx):


Saving sorteio1.xlsx to sorteio1 (1).xlsx
Mounted at /content/drive
Colunas carregadas: Index(['Sorteio', 'Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5',
       'Numero6'],
      dtype='object')
Treinando um novo modelo...
Salvando o modelo treinado no Google Drive...
Acurácia média nas predições: 99.99%
Simulação não acertou em 10.000 tentativas.
Sequência simulada: None


In [1]:
!pip install pandas openpyxl tensorflow


gera 3 sequencias de numeros com base no aprendizado guardado no drive

In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
import joblib
import os
import random
from google.colab import files
from google.colab import drive
import json

# Função para fazer o upload do arquivo no Colab
def selecionar_arquivo():
    print("Selecione o arquivo de sorteios (em formato .xlsx):")
    arquivo = files.upload()
    caminho_arquivo = next(iter(arquivo))  # Pega o primeiro arquivo carregado
    return caminho_arquivo

# Função para montar o Google Drive
def montar_drive():
    drive.mount('/content/drive')
    caminho_modelo_drive = '/content/drive/MyDrive/modelo_sorteio.pkl'  # Caminho no Google Drive
    caminho_sequencias_drive = '/content/drive/MyDrive/sequencias.json'  # Caminho para sequências geradas
    return caminho_modelo_drive, caminho_sequencias_drive

# Função para carregar o modelo do Google Drive
def carregar_modelo(caminho_modelo_drive):
    if os.path.exists(caminho_modelo_drive):
        print("Carregando modelo treinado do Google Drive...")
        return joblib.load(caminho_modelo_drive)
    return None

# Função para salvar o modelo no Google Drive
def salvar_modelo(modelo, caminho_modelo_drive):
    print("Salvando o modelo treinado no Google Drive...")
    joblib.dump(modelo, caminho_modelo_drive)

# Função para carregar sequências geradas
def carregar_sequencias(caminho_sequencias_drive):
    if os.path.exists(caminho_sequencias_drive):
        with open(caminho_sequencias_drive, 'r') as f:
            sequencias = json.load(f)
        print("Sequências previamente geradas carregadas.")
        return sequencias
    return []

# Função para salvar sequências geradas
def salvar_sequencias(sequencias, caminho_sequencias_drive):
    with open(caminho_sequencias_drive, 'w') as f:
        json.dump(sequencias, f)
    print("Sequências geradas salvas no Google Drive.")

# Função para criar features adicionais
def criar_features_adicionais(dados):
    colunas_numeros = ['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']

    # Características estatísticas
    dados['media_numeros'] = dados[colunas_numeros].mean(axis=1)
    dados['std_numeros'] = dados[colunas_numeros].std(axis=1)
    dados['soma_numeros'] = dados[colunas_numeros].sum(axis=1)

    # Diferenças entre números consecutivos
    for i in range(len(colunas_numeros) - 1):
        dados[f'diff_{i+1}'] = dados[colunas_numeros[i+1]] - dados[colunas_numeros[i]]

    # Análise de paridade
    for col in colunas_numeros:
        dados[f'{col}_par'] = dados[col] % 2 == 0
    dados['qtd_pares'] = dados[[f'{col}_par' for col in colunas_numeros]].sum(axis=1)

    return dados

# Função para simulação de Monte Carlo e verificação de sequências não sorteadas
def simular_monte_carlo(dados, sequencias_unicas, num_simulacoes=500000000):
    melhores_sequencias = []
    contagem_simulacoes = 0

    while contagem_simulacoes < num_simulacoes and len(melhores_sequencias) < 3:
        contagem_simulacoes += 1
        simulacao = tuple(sorted(random.sample(range(1, 61), 6)))
        if simulacao not in sequencias_unicas:
            melhores_sequencias.append(simulacao)
            print(f"Sequência não sorteada com alta probabilidade encontrada: {simulacao}")

    if len(melhores_sequencias) < 3:
        print("Não foi possível encontrar três sequências não sorteadas com a quantidade de simulações especificada.")

    return melhores_sequencias

# Função para treinar o modelo ou carregar do Google Drive
def treinar_modelo(dados, X_pca, y, caminho_modelo_drive):
    modelo = carregar_modelo(caminho_modelo_drive)
    if modelo is None:
        print("Treinando um novo modelo...")
        modelo_rf = RandomForestClassifier(n_estimators=200, max_depth=15, min_samples_split=5, min_samples_leaf=2, random_state=42)
        modelo_xgb = xgb.XGBClassifier(n_estimators=200, max_depth=8, learning_rate=0.01, subsample=0.8, colsample_bytree=0.8, eval_metric='logloss', random_state=42)
        modelo_gb = GradientBoostingClassifier(n_estimators=200, max_depth=8, learning_rate=0.01, subsample=0.8, random_state=42)

        modelo = MultiOutputClassifier(estimator=VotingClassifier(estimators=[('rf', modelo_rf), ('xgb', modelo_xgb), ('gb', modelo_gb)], voting='soft'))
        modelo.fit(X_pca, y)

        salvar_modelo(modelo, caminho_modelo_drive)
    return modelo

# Executando o código principal
caminho_arquivo = selecionar_arquivo()

if not caminho_arquivo:
    print("Nenhum arquivo foi selecionado.")
else:
    # Montar o Google Drive para salvar o modelo e sequências
    caminho_modelo_drive, caminho_sequencias_drive = montar_drive()

    # Carregar os dados do arquivo
    dados = pd.read_excel(caminho_arquivo, engine='openpyxl')
    print("Colunas carregadas:", dados.columns)

    if len(dados.columns) > 7:
        dados = dados.iloc[:, :7]

    dados.columns = ['Sorteio', 'Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']

    # Criar features avançadas
    dados = criar_features_adicionais(dados)
    colunas_target = ['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']
    colunas_features = [col for col in dados.columns if col not in colunas_target + ['Sorteio']]

    X = dados[colunas_features]
    y = dados[colunas_target]

    imputer = SimpleImputer(strategy='mean')
    X = imputer.fit_transform(X)

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    pca = PCA(n_components=0.98)
    X_pca = pca.fit_transform(X_scaled)

    # Treinando ou carregando o modelo
    modelo = treinar_modelo(dados, X_pca, y, caminho_modelo_drive)

    # Predições e análise
    predicoes = modelo.predict(X_pca)
    acuracias = [accuracy_score(y.iloc[:, i], predicoes[:, i]) for i in range(y.shape[1])]
    acuracia_media = np.mean(acuracias)
    print(f"Acurácia média nas predições: {acuracia_media * 100:.2f}%")

    # Carregar sequências geradas
    sequencias_geradas = carregar_sequencias(caminho_sequencias_drive)
    sequencias_unicas = set(map(tuple, dados[['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']].values))

    if not sequencias_geradas:  # Se não houver sequências geradas, simule novas
        print("Nenhuma sequência gerada anteriormente, realizando simulação de Monte Carlo...")
        melhores_sequencias = simular_monte_carlo(dados, sequencias_unicas)
        salvar_sequencias(melhores_sequencias, caminho_sequencias_drive)  # Salva as novas sequências geradas
    else:
        print("Utilizando sequências previamente geradas:", sequencias_geradas)
        melhores_sequencias = sequencias_geradas

    print("Melhores sequências não sorteadas com alta probabilidade:", melhores_sequencias)


Selecione o arquivo de sorteios (em formato .xlsx):


Saving sorteio.xlsx to sorteio.xlsx
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Colunas carregadas: Index(['Sorteio', 'Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5',
       'Numero6'],
      dtype='object')
Carregando modelo treinado do Google Drive...
Acurácia média nas predições: 84.16%
Sequências previamente geradas carregadas.
Utilizando sequências previamente geradas: [[5, 8, 37, 40, 41, 42], [1, 7, 14, 45, 47, 59], [1, 22, 42, 49, 50, 51]]
Melhores sequências não sorteadas com alta probabilidade: [[5, 8, 37, 40, 41, 42], [1, 7, 14, 45, 47, 59], [1, 22, 42, 49, 50, 51]]


In [2]:
!pip install tpot
!pip install lightgbm
!pip install h2o
!pip install opencv-python
!pip install scikit-learn
!pip install pandas
!pip install numpy
!pip install dask[dataframe]
!pip install joblib
!pip install openpyxl


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 5.6 MB/s eta 0:00:00
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11939 sha256=38e0fa1660fc2607bc9352a4f1cdbfcfc91593e7a7f6d803c344b3c7ffba1e89
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.8/265.8 MB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.46.0.6-py2.py3-none-any.whl size=265859786 sha256=0308ea2a83a8546b38523bc407e99cd76484a573606cfaa7f01d5466d9d5e59b
  Stored in directory: /root/.cache/pip/wheels/0c/95/eb/b449c73f478dbc4557b80170cf7f2aa6db15862877e9a63536
Successfully built h2o
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 4.5 MB/s eta 0:00:00


Parte 1: Carregar e Salvar a Base de Dados no Google Drive

In [9]:
import pandas as pd
import os
from google.colab import files
from google.colab import drive

def selecionar_arquivo(nome_arquivo):
    """
    Função para selecionar um arquivo .xlsx de sorteios.
    Retorna o caminho do arquivo selecionado.
    """
    print(f"Selecione o {nome_arquivo} (em formato .xlsx):")
    arquivo = files.upload()
    if not arquivo:
        return None
    return next(iter(arquivo))  # Retorna o primeiro arquivo selecionado

def montar_drive():
    """
    Função para montar o Google Drive no Colab.
    Retorna o caminho do diretório onde o modelo será salvo.
    """
    drive.mount('/content/drive')
    caminho_base = '/content/drive/MyDrive/modelo_loteria/'
    if not os.path.exists(caminho_base):
        os.makedirs(caminho_base)
    return caminho_base

def criar_features_adicionais(dados):
    """
    Cria características adicionais baseadas nas colunas de números de sorteio.
    """
    colunas_numeros = ['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']

    # Características estatísticas
    dados['media_numeros'] = dados[colunas_numeros].mean(axis=1)
    dados['std_numeros'] = dados[colunas_numeros].std(axis=1)
    dados['soma_numeros'] = dados[colunas_numeros].sum(axis=1)

    # Diferenças entre números consecutivos
    for i in range(len(colunas_numeros) - 1):
        dados[f'diff_{i+1}'] = dados[colunas_numeros[i+1]] - dados[colunas_numeros[i]]

    # Análise de paridade
    for col in colunas_numeros:
        dados[f'{col}_par'] = dados[col] % 2 == 0
    dados['qtd_pares'] = dados[[f'{col}_par' for col in colunas_numeros]].sum(axis=1)

    return dados

def salvar_arquivo(dados, caminho_base, nome_arquivo):
    """
    Função para salvar o DataFrame 'dados' no Google Drive, sobrescrevendo o arquivo existente.
    """
    caminho_arquivo = os.path.join(caminho_base, nome_arquivo)
    dados.to_excel(caminho_arquivo, index=False, engine='openpyxl')
    print(f"Arquivo salvo com sucesso em: {caminho_arquivo}")

def main():
    """
    Função principal que carrega os dados, realiza o treinamento e testes.
    """
    # Setup inicial
    # Carregar o arquivo em ordem crescente
    caminho_arquivo_crescente = selecionar_arquivo("arquivo com dados em ordem crescente")
    if not caminho_arquivo_crescente:
        print("Nenhum arquivo foi selecionado.")
        return

    # Montar Google Drive
    caminho_base = montar_drive()

    # Carregar e preparar o primeiro arquivo (ordem crescente)
    dados_crescentes = pd.read_excel(caminho_arquivo_crescente, engine='openpyxl')
    print("Colunas do arquivo em ordem crescente carregadas:", dados_crescentes.columns)

    # Garantir que apenas as primeiras 7 colunas sejam usadas
    if len(dados_crescentes.columns) > 7:
        dados_crescentes = dados_crescentes.iloc[:, :7]

    # Renomeando as colunas para um formato padrão
    dados_crescentes.columns = ['Sorteio', 'Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']

    # Criar as características adicionais para o arquivo em ordem crescente
    dados_crescentes = criar_features_adicionais(dados_crescentes)

    # Salvar o arquivo em ordem crescente
    salvar_arquivo(dados_crescentes, caminho_base, 'Base_dados.xlsx')

    # Carregar o segundo arquivo (ordem de sorteio)
    caminho_arquivo_aleatorio = selecionar_arquivo("arquivo com dados em ordem de sorteio")
    if not caminho_arquivo_aleatorio:
        print("Nenhum arquivo foi selecionado.")
        return

    # Carregar e preparar o segundo arquivo (ordem de sorteio)
    dados_aleatorios = pd.read_excel(caminho_arquivo_aleatorio, engine='openpyxl')
    print("Colunas do arquivo em ordem de sorteio carregadas:", dados_aleatorios.columns)

    # Garantir que apenas as primeiras 7 colunas sejam usadas
    if len(dados_aleatorios.columns) > 7:
        dados_aleatorios = dados_aleatorios.iloc[:, :7]

    # Renomeando as colunas para um formato padrão
    dados_aleatorios.columns = ['Sorteio', 'Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']

    # Criar as características adicionais para o arquivo em ordem de sorteio
    dados_aleatorios = criar_features_adicionais(dados_aleatorios)

    # Salvar o arquivo em ordem de sorteio
    salvar_arquivo(dados_aleatorios, caminho_base, 'base_aleatoria.xlsx')

if __name__ == "__main__":
    main()


Selecione o arquivo com dados em ordem crescente (em formato .xlsx):


Saving sorteio.xlsx to sorteio (1).xlsx
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Colunas do arquivo em ordem crescente carregadas: Index(['Sorteio', 'Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5',
       'Numero6'],
      dtype='object')
Arquivo salvo com sucesso em: /content/drive/MyDrive/modelo_loteria/Base_dados.xlsx
Selecione o arquivo com dados em ordem de sorteio (em formato .xlsx):


Saving sorte.xlsx to sorte (1).xlsx
Colunas do arquivo em ordem de sorteio carregadas: Index(['Sorteio', 'Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5',
       'Numero6'],
      dtype='object')
Arquivo salvo com sucesso em: /content/drive/MyDrive/modelo_loteria/base_aleatoria.xlsx


Parte 2: **Treinamento** inicial

In [15]:
import pandas as pd
import joblib
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import cross_val_score

# Função para adaptar os dados
def preparar_dados(X, y):
    """
    Transforma os dados de forma a adequá-los para o treinamento do modelo.
    """
    # Adicione aqui qualquer transformação necessária nos dados
    return X, y

class ModeloPipeline:
    def __init__(self, caminho_modelos):
        self.imputer = SimpleImputer(strategy='mean')
        self.scaler = MinMaxScaler()
        self.pca = PCA(n_components=0.98)
        self.modelo = None
        self.colunas_features = None
        self.colunas_target = None
        self.historico_score = []
        self.caminho_modelos = caminho_modelos

        # Carregar transformações e modelos, se existirem
        self.carregar_modelos()

    def carregar_modelos(self):
        """
        Carrega modelos e transformações salvas, se disponíveis.
        """
        if os.path.exists(os.path.join(self.caminho_modelos, 'modelo_completo.pkl')):
            self.modelo = joblib.load(os.path.join(self.caminho_modelos, 'modelo_completo.pkl'))
        if os.path.exists(os.path.join(self.caminho_modelos, 'modelo_pca.pkl')):
            self.pca = joblib.load(os.path.join(self.caminho_modelos, 'modelo_pca.pkl'))
        if os.path.exists(os.path.join(self.caminho_modelos, 'modelo_scaler.pkl')):
            self.scaler = joblib.load(os.path.join(self.caminho_modelos, 'modelo_scaler.pkl'))
        if os.path.exists(os.path.join(self.caminho_modelos, 'modelo_imputer.pkl')):
            self.imputer = joblib.load(os.path.join(self.caminho_modelos, 'modelo_imputer.pkl'))

    def fit(self, X, y):
        """
        Treina o modelo com os dados fornecidos.
        """
        # Guardar as colunas de features e alvos
        self.colunas_features = X.columns.tolist() if isinstance(X, pd.DataFrame) else None
        self.colunas_target = y.columns.tolist() if isinstance(y, pd.DataFrame) else None

        # Pipeline de transformação
        X_imputed = self.imputer.fit_transform(X)
        X_scaled = self.scaler.fit_transform(X_imputed)
        X_pca = self.pca.fit_transform(X_scaled)

        # Modelos básicos para MultiOutputClassifier
        modelo_gb = GradientBoostingClassifier(n_estimators=200, max_depth=8, learning_rate=0.01, subsample=0.8, random_state=42)

        # Substituição do LGBMClassifier por RandomForestClassifier
        modelo_rf = RandomForestClassifier(n_estimators=150, max_depth=10, random_state=42)

        # Usando MultiOutputClassifier com VotingClassifier
        self.modelo = MultiOutputClassifier(
            estimator=VotingClassifier(
                estimators=[('gb', modelo_gb), ('rf', modelo_rf)],
                voting='soft'
            )
        )
        self.modelo.fit(X_pca, y)
        return self

    def save_model(self):
        """
        Salva o modelo treinado e os modelos de transformação.
        """
        # Salvar o modelo final
        joblib.dump(self.modelo, os.path.join(self.caminho_modelos, 'modelo_completo.pkl'))
        joblib.dump(self.pca, os.path.join(self.caminho_modelos, 'modelo_pca.pkl'))
        joblib.dump(self.scaler, os.path.join(self.caminho_modelos, 'modelo_scaler.pkl'))
        joblib.dump(self.imputer, os.path.join(self.caminho_modelos, 'modelo_imputer.pkl'))
        print("Modelos salvos com sucesso!")

    def plot_score_evolution(self):
        """
        Plota a evolução dos scores durante o treinamento.
        """
        plt.figure(figsize=(10, 6))
        scores = [score for _, score in self.historico_score]
        plt.plot(self.colunas_target, scores, marker='o', color='b', linestyle='--')
        plt.xlabel('Colunas Alvo')
        plt.ylabel('Score')
        plt.title('Evolução dos Scores dos Modelos')
        plt.grid()
        plt.show()

def main():
    # Carregar dados e treinar o modelo
    dados = pd.read_excel("/content/drive/MyDrive/modelo_loteria/Base_dados.xlsx", engine='openpyxl')
    colunas_target = ['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']
    colunas_features = [col for col in dados.columns if col not in colunas_target + ['Sorteio']]

    X = dados[colunas_features]
    y = dados[colunas_target]

    # Preparar os dados
    X, y = preparar_dados(X, y)

    # Treinar o modelo
    caminho_base = "/content/drive/MyDrive/modelo_loteria/"
    modelo = ModeloPipeline(caminho_base)
    modelo.fit(X, y)

    # Salvar o modelo treinado
    modelo.save_model()

    # Plotar a evolução dos scores
    modelo.plot_score_evolution()

if __name__ == "__main__":
    main()


KeyboardInterrupt: 

Parte 2.1 - aleatorios

In [ ]:
#Parte 2.1 - aleatorios
import pandas as pd
import joblib
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from tpot import TPOTClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
import os
import lightgbm as lgb
from tqdm import tqdm

class ModeloPipelineAtualizado:
    def __init__(self):
        # Carregar modelos e transformações previamente salvos
        self.imputer = joblib.load('/content/drive/MyDrive/modelo_loteria/modelo_imputer.pkl')
        self.scaler = joblib.load('/content/drive/MyDrive/modelo_loteria/modelo_scaler.pkl')
        self.pca = joblib.load('/content/drive/MyDrive/modelo_loteria/modelo_pca.pkl')
        self.modelo = joblib.load('/content/drive/MyDrive/modelo_loteria/modelo_completo.pkl')
        self.modelos_tpot = {}  # Para armazenar os modelos TPOT atualizados

    def atualizar_pipeline(self, X_novos, y_novos):
        """
        Função para atualizar a pipeline com novos dados (dados aleatórios) e treinar novamente.
        """
        # Transformar os novos dados
        X_imputed = self.imputer.transform(X_novos)
        X_scaled = self.scaler.transform(X_imputed)
        X_pca = self.pca.transform(X_scaled)

        # Treinar novamente o modelo TPOT para cada coluna alvo
        for coluna_alvo in tqdm(y_novos.columns, desc="Atualizando TPOT para cada alvo"):
            _, modelo_tpot = self.fit_tpot(X_pca, y_novos[coluna_alvo], coluna_alvo)
            self.modelos_tpot[coluna_alvo] = modelo_tpot

        # Usar MultiOutputClassifier com VotingClassifier para re-treinar o modelo principal
        modelo_gb = GradientBoostingClassifier(n_estimators=200, max_depth=8, learning_rate=0.01, subsample=0.8, random_state=42)
        modelo_lgbm = lgb.LGBMClassifier(n_estimators=200, max_depth=8, scale_pos_weight=10)

        self.modelo = MultiOutputClassifier(
            estimator=VotingClassifier(
                estimators=[('gb', modelo_gb), ('lgbm', modelo_lgbm)],
                voting='soft'
            )
        )
        self.modelo.fit(X_pca, y_novos)

    def fit_tpot(self, X, y_coluna, coluna_alvo):
        """
        Função auxiliar para treinar TPOT em uma coluna alvo específica.
        """
        custom_config = {
            'sklearn.naive_bayes.GaussianNB': {},
            'sklearn.tree.DecisionTreeClassifier': {'max_depth': [5, 10, None]},
            'sklearn.ensemble.RandomForestClassifier': {
                'n_estimators': [10, 50],
                'max_features': ['auto', 'sqrt', 'log2']
            },
            'sklearn.linear_model.LogisticRegression': {
                'penalty': ['l2'],
                'C': [1.0, 0.5, 0.1]
            }
        }

        modelo_tpot = TPOTClassifier(
            generations=3,
            population_size=10,
            verbosity=2,
            config_dict=custom_config,
            random_state=42,
            n_jobs=-1
        )
        modelo_tpot.fit(X, y_coluna)
        return coluna_alvo, modelo_tpot

    def save_model(self, caminho):
        """
        Salva o modelo atualizado e os modelos TPOT separadamente, assim como as transformações.
        """
        # Salvar o modelo final
        joblib.dump(self.modelo, os.path.join(caminho, 'modelo_completo_atualizado.pkl'))
        # Salvar cada modelo TPOT separadamente
        for coluna_alvo, modelo_tpot in self.modelos_tpot.items():
            joblib.dump(modelo_tpot, os.path.join(caminho, f'modelo_tpot_{coluna_alvo}_atualizado.pkl'))
        # Salvar transformações
        joblib.dump(self.pca, os.path.join(caminho, 'modelo_pca_atualizado.pkl'))
        joblib.dump(self.scaler, os.path.join(caminho, 'modelo_scaler_atualizado.pkl'))
        joblib.dump(self.imputer, os.path.join(caminho, 'modelo_imputer_atualizado.pkl'))
        print("Modelos atualizados e salvos com sucesso!")

def main_atualizacao():
    """
    Função principal para carregar dados aleatórios, atualizar a pipeline e salvar o modelo atualizado.
    """
    # Carregar dados aleatórios
    dados_aleatorios = pd.read_excel("/content/drive/MyDrive/modelo_loteria/base_aleatoria.xlsx", engine='openpyxl')
    colunas_target = ['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']
    colunas_features = [col for col in dados_aleatorios.columns if col not in colunas_target + ['Sorteio']]

    X_aleatorios = dados_aleatorios[colunas_features]
    y_aleatorios = dados_aleatorios[colunas_target]

    # Carregar a pipeline já treinada
    modelo_atualizado = ModeloPipelineAtualizado()

    # Atualizar a pipeline com os novos dados
    modelo_atualizado.atualizar_pipeline(X_aleatorios, y_aleatorios)

    # Salvar o modelo atualizado
    caminho_base = "/content/drive/MyDrive/modelo_loteria/"
    if not os.path.exists(caminho_base):
        os.makedirs(caminho_base)

    modelo_atualizado.save_model(caminho_base)

if __name__ == "__main__":
    main_atualizacao()


**Aplicador** de **Restrições**

In [11]:
import pandas as pd
import joblib
import os

# Função para aplicar as restrições de números por posição
def aplicar_restricoes(predicoes):
    """
    Aplica restrições específicas em cada posição do vetor de números sorteados.
    """
    restricoes = {
        0: [40, 42, 44, 45, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
        1: [1, 49, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60],
        2: [1, 2, 55, 57, 58, 59, 60],
        3: [1, 2, 3, 4, 5, 6, 7, 59, 60],
        4: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 60],
        5: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23]
    }

    predicoes_filtradas = []
    for predicao in predicoes:
        atende_restricoes = True
        for i, numero in enumerate(predicao):
            if numero in restricoes[i]:
                atende_restricoes = False
                break
        if atende_restricoes:
            predicoes_filtradas.append(predicao)
    return predicoes_filtradas

# Função para carregar o modelo e aplicar as restrições às previsões
def gerar_previsoes_filtradas(caminho_base, X):
    # Carregar o modelo salvo e as transformações
    modelo = joblib.load(os.path.join(caminho_base, 'modelo_completo.pkl'))
    scaler = joblib.load(os.path.join(caminho_base, 'modelo_scaler.pkl'))
    imputer = joblib.load(os.path.join(caminho_base, 'modelo_imputer.pkl'))
    pca = joblib.load(os.path.join(caminho_base, 'modelo_pca.pkl'))

    # Pré-processar os dados de entrada
    X_imputed = imputer.transform(X)
    X_scaled = scaler.transform(X_imputed)
    X_pca = pca.transform(X_scaled)

    # Gerar previsões
    predicoes = modelo.predict(X_pca)

    # Aplicar restrições às previsões
    predicoes_filtradas = aplicar_restricoes(predicoes)

    return predicoes_filtradas

# Função principal para carregar os dados e processar as previsões
def main():
    # Carregar a base de dados para obter as features
    dados = pd.read_excel("/content/drive/MyDrive/modelo_loteria/Base_dados.xlsx", engine='openpyxl')
    colunas_target = ['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']
    colunas_features = [col for col in dados.columns if col not in colunas_target + ['Sorteio']]

    X = dados[colunas_features]

    # Gerar previsões aplicando as restrições
    caminho_base = "/content/drive/MyDrive/modelo_loteria/"
    predicoes_filtradas = gerar_previsoes_filtradas(caminho_base, X)

    # Exibir previsões finais
    for i, predicao in enumerate(predicoes_filtradas):
        print(f"Previsão {i+1}: {predicao}")

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/modelo_loteria/modelo_scaler.pkl'

Parte 3 - Carregamento e Incremento

In [14]:
# Parte 3 - Carregamento e Incremento
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from tqdm import tqdm  # Importa a biblioteca tqdm para o contador de progresso

class ModeloPipeline:
    def __init__(self, caminho_modelo):
        # Carregar os modelos salvos, exceto TPOT (será treinado separadamente)
        self.modelo = joblib.load(caminho_modelo + '/modelo_completo.pkl')
        self.pca = joblib.load(caminho_modelo + '/modelo_pca.pkl')
        self.scaler = joblib.load(caminho_modelo + '/modelo_scaler.pkl')
        self.imputer = joblib.load(caminho_modelo + '/modelo_imputer.pkl')
        print("Modelos carregados com sucesso!")

    def incremental_fit(self, X, y):
        # Preparar os dados de entrada
        X_imputed = self.imputer.transform(X)
        X_scaled = self.scaler.transform(X_imputed)
        X_pca = self.pca.transform(X_scaled)

        # Dividir os dados em pequenos lotes para o treinamento incremental (caso queira realizar em lotes)
        num_batches = len(X) // 100  # Defina o número de lotes conforme necessário (100 é um exemplo)
        for i in tqdm(range(num_batches), desc="Treinando incrementalmente", ncols=100):
            # Pegue um lote dos dados para o treinamento
            batch_X = X_pca[i*100 : (i+1)*100]
            batch_y = y[i*100 : (i+1)*100]

            # Re-treinar o VotingClassifier com os dados do lote
            self.modelo.fit(batch_X, batch_y)

        print("Modelo incremental treinado com sucesso!")

    def save_model(self, caminho):
        joblib.dump(self.modelo, caminho + '/modelo_completo.pkl')
        print("Modelos incrementais salvos com sucesso!")

def main():
    # Carregar os dados novos para incremento
    dados = pd.read_excel("Base_dados.xlsx", engine='openpyxl')
    colunas_features = ['feature1', 'feature2', 'feature3']  # Exemplo
    colunas_target = ['target1', 'target2']  # Exemplo

    X = dados[colunas_features]
    y = dados[colunas_target]

    caminho_base = "/content/drive/MyDrive/modelo_loteria/"

    # Carregar o modelo treinado previamente
    modelo = ModeloPipeline(caminho_base)

    # Incrementar o modelo com os dados novos
    modelo.incremental_fit(X, y)

    # Salvar o modelo incremental
    modelo.save_model(caminho_base)

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'Base_dados.xlsx'

Parte 4[texto do link](https://) - Treinamento Exclusivo do TPOT

In [ ]:
# Parte 4 - Treinamento Exclusivo do TPOT
import joblib
import pandas as pd
from tpot import TPOTClassifier
from tqdm import tqdm  # Importa a biblioteca tqdm para o contador de progresso

class TPOTWithProgress:
    def __init__(self, max_time_mins=60, generations=5, population_size=20):
        """
        Inicializa o TPOT com parâmetros e a capacidade de mostrar progresso.
        :param max_time_mins: Tempo máximo de treinamento em minutos
        :param generations: Número de gerações para evolução
        :param population_size: Tamanho da população em cada geração
        """
        self.max_time_mins = max_time_mins
        self.generations = generations
        self.population_size = population_size

    def fit(self, X, y):
        # Cria um modelo TPOT
        self.model = TPOTClassifier(
            generations=self.generations,
            population_size=self.population_size,
            verbosity=2,  # Controla o nível de detalhes do processo
            max_time_mins=self.max_time_mins
        )

        # Cria uma barra de progresso
        total_steps = self.generations * self.population_size  # Estimativa do número de etapas
        with tqdm(total=total_steps, desc="Treinando TPOT", ncols=100) as pbar:
            # O TPOT normalmente não possui suporte direto para progresso, então vamos usar a função fit diretamente
            # Esta função tentará monitorar o progresso por meio da evolução da população e número de gerações
            def callback(step):
                pbar.update(1)  # Atualiza a barra de progresso a cada passo
            self.model.fit(X, y, callback=callback)

        print("Modelo TPOT treinado com sucesso!")

    def save(self, caminho):
        # Salvar o modelo treinado
        joblib.dump(self.model, caminho + '/modelo_tpot_atualizado.pkl')
        print("Modelo TPOT salvo com sucesso!")

def main():
    # Carregar os dados novos para treinamento do TPOT
    dados = pd.read_excel("novos_dados.xlsx", engine='openpyxl')
    colunas_features = ['feature1', 'feature2', 'feature3']  # Exemplo
    colunas_target = ['target1', 'target2']  # Exemplo

    X = dados[colunas_features]
    y = dados[colunas_target]

    caminho_base = "/content/drive/MyDrive/modelo_loteria/"

    # Carregar o modelo TPOT previamente treinado
    modelo_tpot = TPOTWithProgress()

    # Re-treinar o modelo TPOT
    modelo_tpot.fit(X, y)

    # Salvar o modelo TPOT treinado
    modelo_tpot.save(caminho_base)

if __name__ == "__main__":
    main()


Parte Final - Gerador de Sequência

In [ ]:
import pandas as pd
import joblib
import os
import numpy as np

# Função para aplicar as restrições de números por posição
def aplicar_restricoes(predicoes):
    """
    Aplica restrições específicas em cada posição do vetor de números sorteados.
    """
    restricoes = {
        0: [40, 42, 44, 45, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
        1: [1, 49, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60],
        2: [1, 2, 55, 57, 58, 59, 60],
        3: [1, 2, 3, 4, 5, 6, 7, 59, 60],
        4: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 60],
        5: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23]
    }

    predicoes_filtradas = []
    for predicao in predicoes:
        atende_restricoes = True
        for i, numero in enumerate(predicao):
            if numero in restricoes[i]:
                atende_restricoes = False
                break
        if atende_restricoes:
            predicoes_filtradas.append(predicao)
    return predicoes_filtradas

# Função para calcular a probabilidade de acerto de uma sequência com base na frequência
def calcular_probabilidade_acerto(sequencia, dados_base):
    """
    Calcula a probabilidade de acerto de uma sequência com base na frequência dos números.
    """
    frequencia_numeros = {i: [] for i in range(6)}

    # Contabiliza a frequência de cada número em cada posição
    for _, row in dados_base.iterrows():
        for i in range(6):
            frequencia_numeros[i].append(row[f'Numero{i+1}'])

    # Calcula a frequência de cada número em cada posição
    probabilidade = 1
    for i, numero in enumerate(sequencia):
        freq = frequencia_numeros[i].count(numero) / len(dados_base)  # Frequência na base
        probabilidade *= freq  # Multiplicando as probabilidades

    return probabilidade

# Função para calcular a deviância de uma sequência em relação aos sorteios passados
def calcular_deviancia(sequencia, dados_base):
    """
    Calcula a deviância de uma sequência com base nas sequências passadas.
    A deviância é a diferença entre a sequência gerada e as sequências anteriores.
    """
    menores_distancias = []
    for _, row in dados_base.iterrows():
        distancia = sum([1 if sequencia[i] != row[f'Numero{i+1}'] else 0 for i in range(6)])
        menores_distancias.append(distancia)

    # A distância mínima é considerada uma medida de quão "próxima" a sequência está de um sorteio passado
    deviancia_media = np.mean(menores_distancias)
    return deviancia_media

# Função para carregar o modelo e aplicar as restrições às previsões
def gerar_previsoes_filtradas(caminho_base, X):
    # Carregar o modelo salvo e as transformações
    modelo = joblib.load(os.path.join(caminho_base, 'modelo_completo.pkl'))
    scaler = joblib.load(os.path.join(caminho_base, 'modelo_scaler.pkl'))
    imputer = joblib.load(os.path.join(caminho_base, 'modelo_imputer.pkl'))
    pca = joblib.load(os.path.join(caminho_base, 'modelo_pca.pkl'))

    # Pré-processar os dados de entrada
    X_imputed = imputer.transform(X)
    X_scaled = scaler.transform(X_imputed)
    X_pca = pca.transform(X_scaled)

    # Gerar previsões
    predicoes = modelo.predict(X_pca)

    # Aplicar restrições às previsões
    predicoes_filtradas = aplicar_restricoes(predicoes)

    return predicoes_filtradas

# Função para verificar se uma sequência já foi jogada
def verificar_sequencia_existe(sequencia, dados_base):
    """
    Verifica se a sequência gerada já foi jogada anteriormente.
    """
    for index, row in dados_base.iterrows():
        if all(num in row.values for num in sequencia):
            return True
    return False

# Função para gerar sequências de 6 números com maior chance de acerto
def gerar_sequencias_com_chances(caminho_base, dados_base):
    # Carregar a base de dados
    X = dados_base.drop(columns=['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6'])

    # Gerar previsões
    predicoes_filtradas = gerar_previsoes_filtradas(caminho_base, X)

    # Filtrar sequências que ainda não foram jogadas e respeitam as restrições
    sequencias_validas = []
    for predicao in predicoes_filtradas:
        if not verificar_sequencia_existe(predicao, dados_base):
            # Calcular a probabilidade de acerto e deviância
            probabilidade = calcular_probabilidade_acerto(predicao, dados_base)
            deviancia = calcular_deviancia(predicao, dados_base)

            # Se a sequência tiver uma boa chance de acerto, adicione à lista
            if probabilidade > 0.01 and deviancia < 3:  # Exemplo de condições para filtrar boas sequências
                sequencias_validas.append(predicao)

        # Se já tivermos 3 sequências válidas, interrompemos
        if len(sequencias_validas) >= 3:
            break

    return sequencias_validas

# Função principal
def main():
    # Carregar a base de dados histórica para garantir que as sequências não se repitam
    caminho_base = "/content/drive/MyDrive/modelo_loteria/"
    dados_base = pd.read_excel("/content/drive/MyDrive/modelo_loteria/Base_dados.xlsx", engine='openpyxl')

    # Gerar 3 sequências com maior chance de acerto que não foram jogadas
    sequencias = gerar_sequencias_com_chances(caminho_base, dados_base)

    # Exibir as sequências geradas
    print("Sequências geradas com maior chance de acerto:")
    for i, sequencia in enumerate(sequencias):
        print(f"Sequência {i+1}: {sequencia}")

if __name__ == "__main__":
    main()


In [26]:
import pandas as pd
import numpy as np
import joblib
import os
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from sklearn.multioutput import MultiOutputClassifier
import lightgbm as lgb
from tpot import TPOTClassifier
import h2o
from h2o.estimators import H2ORandomForestEstimator
from google.colab import files
from google.colab import drive

# Inicializando o H2O
h2o.init()

class ModeloPipeline:
    def __init__(self):
        self.imputer = SimpleImputer(strategy='mean')
        self.scaler = MinMaxScaler()
        self.pca = PCA(n_components=0.98)
        self.modelo = None
        self.colunas_features = None
        self.colunas_target = None
        self.modelo_tpot = None  # Inicializando o modelo TPOT como None
        self.modelo_h2o = None

    def fit(self, X, y):
        # Guardar colunas
        if isinstance(X, pd.DataFrame):
            self.colunas_features = X.columns.tolist()
        if isinstance(y, pd.DataFrame):
            self.colunas_target = y.columns.tolist()

        # Pipeline de transformação
        X_imputed = self.imputer.fit_transform(X)
        X_scaled = self.scaler.fit_transform(X_imputed)
        X_pca = self.pca.fit_transform(X_scaled)

        # Treinar múltiplos modelos
        modelo_gb = GradientBoostingClassifier(n_estimators=200, max_depth=8, learning_rate=0.01, subsample=0.8, random_state=42)
        modelo_lgbm = lgb.LGBMClassifier(n_estimators=200, max_depth=8)

        # Usar TPOT para treinar o modelo
        self.modelo_tpot = TPOTClassifier(verbosity=2, random_state=42)  # Inicializa o TPOT aqui
        self.modelo_tpot.fit(X, y)

        # Treinando H2O RandomForest
        train_h2o = h2o.H2OFrame(pd.concat([X, y], axis=1))
        self.modelo_h2o = H2ORandomForestEstimator(ntrees=200, max_depth=15)
        self.modelo_h2o.train(x=self.colunas_features, y=self.colunas_target[0], training_frame=train_h2o)

        # Usar MultiOutputClassifier com VotingClassifier
        self.modelo = MultiOutputClassifier(
            estimator=VotingClassifier(
                estimators=[('gb', modelo_gb), ('lgbm', modelo_lgbm)],
                voting='soft'
            )
        )
        self.modelo.fit(X_pca, y)
        return self

    def incremental_fit(self, X, y):
        # Verifica se o modelo tpot já foi inicializado corretamente
        if self.modelo_tpot is None:
            print("Inicializando TPOTClassifier novamente...")
            self.modelo_tpot = TPOTClassifier(verbosity=2, random_state=42)  # Inicializa se necessário

        # Re-treina o modelo com os dados adicionais
        X_imputed = self.imputer.transform(X)
        X_scaled = self.scaler.transform(X_imputed)
        X_pca = self.pca.transform(X_scaled)

        self.modelo_tpot.fit(X, y)  # Re-treinando o TPOT com dados novos

        # Re-treina o modelo H2O RandomForest
        train_h2o = h2o.H2OFrame(pd.concat([X, y], axis=1))
        if self.modelo_h2o is None:
            self.modelo_h2o = H2ORandomForestEstimator(ntrees=200, max_depth=15)

        self.modelo_h2o.train(x=self.colunas_features, y=self.colunas_target[0], training_frame=train_h2o)

        # Re-treina o VotingClassifier
        self.modelo.fit(X_pca, y)
        return self

    def transform_input(self, X):
        if isinstance(X, pd.DataFrame):
            X = X[self.colunas_features]
        X_imputed = self.imputer.transform(X)
        X_scaled = self.scaler.transform(X_imputed)
        X_pca = self.pca.transform(X_scaled)
        return X_pca

    def predict(self, X):
        X_pca = self.transform_input(X)
        return self.modelo.predict(X_pca)

def selecionar_arquivo():
    print("Selecione o arquivo de sorteios (em formato .xlsx):")
    arquivo = files.upload()
    return next(iter(arquivo))

def montar_drive():
    drive.mount('/content/drive')
    return '/content/drive/MyDrive/modelo_loteria/'

def criar_features_adicionais(dados):
    colunas_numeros = ['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']

    # Características estatísticas
    dados['media_numeros'] = dados[colunas_numeros].mean(axis=1)
    dados['std_numeros'] = dados[colunas_numeros].std(axis=1)
    dados['soma_numeros'] = dados[colunas_numeros].sum(axis=1)

    # Diferenças entre números consecutivos
    for i in range(len(colunas_numeros) - 1):
        dados[f'diff_{i+1}'] = dados[colunas_numeros[i+1]] - dados[colunas_numeros[i]]

    # Análise de paridade
    for col in colunas_numeros:
        dados[f'{col}_par'] = dados[col] % 2 == 0
    dados['qtd_pares'] = dados[[f'{col}_par' for col in colunas_numeros]].sum(axis=1)

    return dados

def salvar_pipeline(pipeline, caminho_base):
    if not os.path.exists(caminho_base):
        os.makedirs(caminho_base)

    joblib.dump(pipeline, os.path.join(caminho_base, 'pipeline_completa.pkl'))

    # Salvar informações das colunas
    info_colunas = {
        'features': pipeline.colunas_features,
        'target': pipeline.colunas_target
    }
    with open(os.path.join(caminho_base, 'info_colunas.json'), 'w') as f:
        json.dump(info_colunas, f)

def carregar_pipeline(caminho_base):
    if not os.path.exists(caminho_base):
        return None

    caminho_pipeline = os.path.join(caminho_base, 'pipeline_completa.pkl')
    if not os.path.exists(caminho_pipeline):
        return None

    return joblib.load(caminho_pipeline)

def main():
    # Setup inicial
    caminho_arquivo = selecionar_arquivo()
    if not caminho_arquivo:
        print("Nenhum arquivo foi selecionado.")
        return

    caminho_base = montar_drive()

    # Carregar e preparar dados
    dados = pd.read_excel(caminho_arquivo, engine='openpyxl')
    print("Colunas carregadas:", dados.columns)

    if len(dados.columns) > 7:
        dados = dados.iloc[:, :7]

    dados.columns = ['Sorteio', 'Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']
    dados = criar_features_adicionais(dados)

    colunas_target = ['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']
    colunas_features = [col for col in dados.columns if col not in colunas_target + ['Sorteio']]

    X = dados[colunas_features]
    y = dados[colunas_target]

    # Carregar ou criar pipeline
    pipeline = carregar_pipeline(caminho_base)
    if pipeline is None:
        print("Criando nova pipeline...")
        pipeline = ModeloPipeline()
        pipeline.fit(X, y)
        salvar_pipeline(pipeline, caminho_base)
    else:
        print("Incrementando o modelo com dados novos...")
        pipeline.incremental_fit(X, y)
        salvar_pipeline(pipeline, caminho_base)

    # Variáveis de contagem
    tentativas_ate_acerto = 0
    previsoes_totais = 0
    total_sequencias = len(dados)

    # Teste de predição
    print("\nTestando predições...")
    for i in range(min(5, len(dados))):
        entrada = X.iloc[[i]]
        predicao = pipeline.predict(entrada)
        real = y.iloc[i].values
        previsoes_totais += 1
        tentativas_ate_acerto += 1

        acerto = np.array_equal(predicao[0], real)

        print(f"\nSequência {i+1}:")
        print(f"Predição: {predicao[0]}")
        print(f"Real: {real}")
        print(f"Acerto: {acerto}")

        # Verifica se acertou na primeira tentativa
        if acerto and tentativas_ate_acerto == 1:
            print("Acerto na primeira tentativa!")
        else:
            print(f"Tentativas até o acerto: {tentativas_ate_acerto}")
        print("-" * 50)

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'seu_arquivo.xlsx'

In [43]:
# Importações necessárias
import os
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive, files
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.multioutput import MultiOutputClassifier
from tpot import TPOTClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm
import concurrent.futures

# Montar o Google Drive
drive.mount('/content/drive')
caminho_base = '/content/drive/MyDrive/modelo_loteria/'
if not os.path.exists(caminho_base):
    os.makedirs(caminho_base)

def carregar_dados(tipo='crescente'):
    print(f"Selecione o arquivo de dados {tipo} (formato .xlsx):")
    arquivo = files.upload()
    if not arquivo:
        return None
    caminho_arquivo = next(iter(arquivo))
    dados = pd.read_excel(caminho_arquivo, engine='openpyxl')
    return dados

def criar_features_adicionais(dados):
    colunas_numeros = ['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']
    dados['media_numeros'] = dados[colunas_numeros].mean(axis=1)
    dados['std_numeros'] = dados[colunas_numeros].std(axis=1)
    dados['soma_numeros'] = dados[colunas_numeros].sum(axis=1)
    for i in range(len(colunas_numeros) - 1):
        dados[f'diff_{i+1}'] = dados[colunas_numeros[i+1]] - dados[colunas_numeros[i]]
    for col in colunas_numeros:
        dados[f'{col}_par'] = dados[col] % 2 == 0
    dados['qtd_pares'] = dados[[f'{col}_par' for col in colunas_numeros]].sum(axis=1)
    return dados

class ModeloPrincipal:
    def __init__(self):
        self.pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
            ('classifier', MultiOutputClassifier(RandomForestClassifier(n_estimators=100)))
        ])

    def treinar(self, X, y):
        self.pipeline.fit(X, y)

    def prever(self, X):
        return self.pipeline.predict(X)

    def salvar(self, caminho):
        joblib.dump(self.pipeline, caminho)

    def carregar(self, caminho):
        self.pipeline = joblib.load(caminho)

class ModeloTPOT:
    def __init__(self):
        self.modelo = TPOTClassifier(generations=5, population_size=20, verbosity=2, random_state=42)

    def treinar(self, X, y):
        self.modelo.fit(X, y)

    def prever(self, X):
        return self.modelo.predict(X)

    def salvar(self, caminho):
        self.modelo.export(caminho)

class ModeloTensorFlow:
    def __init__(self, input_shape):
        self.modelo = Sequential([
            Dense(64, activation='relu', input_shape=input_shape),
            Dropout(0.2),
            Dense(32, activation='relu'),
            Dropout(0.2),
            Dense(6, activation='linear')
        ])
        self.modelo.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    def treinar(self, X, y):
        self.modelo.fit(X, y, epochs=50, batch_size=32, verbose=0)

    def prever(self, X):
        return self.modelo.predict(X)

    def salvar(self, caminho):
        self.modelo.save(caminho)

    def carregar(self, caminho):
        self.modelo = tf.keras.models.load_model(caminho)

def treinamento_incremental(modelo, X, y, batch_size=1000):
    total_batches = len(range(0, len(X), batch_size))
    for i in tqdm(range(0, len(X), batch_size), desc="Treinamento Incremental", total=total_batches):
        X_batch = X[i:i+batch_size]
        y_batch = y[i:i+batch_size]

        # Verificar se y_batch é 1D ou 2D e treinar de acordo
        if y_batch.ndim == 1:  # Se for 1D, não precisamos fatiar
            modelo.treinar(X_batch, y_batch)
        else:  # Se for 2D, treinar para cada coluna
            for idx in range(y_batch.shape[1]):
                modelo.treinar(X_batch, y_batch.iloc[:, idx])  # Usando iloc para acessar colunas

    return modelo

def gerar_mapa_calor(dados):
    plt.figure(figsize=(12, 10))
    sns.heatmap(dados[['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']].corr(),
                annot=True, cmap='coolwarm')
    plt.title('Mapa de Calor das Correlações entre Números')
    plt.savefig(os.path.join(caminho_base, 'mapa_calor.png'))
    plt.close()

def main():
    # Carregar dados
    dados_crescentes = carregar_dados('crescente')
    dados_aleatorios = carregar_dados('aleatório')

    # Processar dados
    dados_crescentes = criar_features_adicionais(dados_crescentes)
    dados_aleatorios = criar_features_adicionais(dados_aleatorios)

    # Preparar dados para treinamento
    X = dados_crescentes.drop(['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6'], axis=1)
    y = dados_crescentes[['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']]

    # Inicializar modelos
    modelo_principal = ModeloPrincipal()
    modelo_tpot = ModeloTPOT()
    modelo_tf = ModeloTensorFlow(input_shape=(X.shape[1],))

    # Treinamento paralelo
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_principal = executor.submit(treinamento_incremental, modelo_principal, X, y)
        future_tpot = executor.submit(treinamento_incremental, modelo_tpot, X, y)
        future_tf = executor.submit(treinamento_incremental, modelo_tf, X, y)

        modelo_principal = future_principal.result()
        modelo_tpot = future_tpot.result()
        modelo_tf = future_tf.result()

    # Salvar modelos
    modelo_principal.salvar(os.path.join(caminho_base, 'modelo_principal.pkl'))
    modelo_tpot.salvar(os.path.join(caminho_base, 'modelo_tpot.py'))
    modelo_tf.salvar(os.path.join(caminho_base, 'modelo_tf'))

    # Gerar mapa de calor
    gerar_mapa_calor(dados_crescentes)

    # Gerar previsões
    X_teste = X.iloc[-1000:]  # Usando as últimas 1000 entradas como teste
    previsoes_principal = modelo_principal.prever(X_teste)
    previsoes_tpot = modelo_tpot.prever(X_teste)
    previsoes_tf = modelo_tf.prever(X_teste)

    # Consolidar previsões (linha corrigida)
    previsoes_consolidadas = np.mean([previsoes_principal, previsoes_tpot, previsoes_tf], axis=0)

    # Mostrar as 3 sequências com maior chance de acerto
    sequencias_finais = np.argsort(previsoes_consolidadas.sum(axis=1))[-3:]
    print("As 3 sequências com maior chance de acerto são:")
    for seq in sequencias_finais:
        print(previsoes_consolidadas[seq])

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Selecione o arquivo de dados crescente (formato .xlsx):


Saving sorteio.xlsx to sorteio (17).xlsx
Selecione o arquivo de dados aleatório (formato .xlsx):


Saving sorte.xlsx to sorte (6).xlsx


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
Treinamento Incremental:   0%|          | 0/3 [00:00<?, ?it/s]

Treinamento Incremental:   0%|          | 0/3 [00:00<?, ?it/s]

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]

...



Treinamento Incremental:  67%|██████▋   | 2/3 [2:59:01<1:06:51, 4011.91s/it]


Generation 2 - Current best internal CV score: 0.26670647241461665




Treinamento Incremental:  67%|██████▋   | 2/3 [3:03:05<1:06:51, 4011.91s/it]


Generation 3 - Current best internal CV score: 0.27687286044104764


KeyboardInterrupt: 

# Nova seção

In [8]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import joblib
import os
import random
from google.colab import drive

# Importando bibliotecas de ML
import tensorflow as tf
import lightgbm as lgb
from tpot import TPOTClassifier
import h2o
from h2o.estimators import H2ORandomForestEstimator
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import VotingClassifier

# Inicializando o H2O
h2o.init()

# Função para montar o Google Drive
def montar_drive():
    drive.mount('/content/drive')
    caminho_base = '/content/drive/MyDrive/modelo_loteria/'  # Caminho para os modelos e sequências
    return caminho_base

# Função para carregar o pipeline
def carregar_pipeline(caminho_base):
    caminho_pipeline = os.path.join(caminho_base, 'pipeline_completa.pkl')
    if os.path.exists(caminho_pipeline):
        print("Carregando o pipeline treinado...")
        return joblib.load(caminho_pipeline)
    return None

# Função para criar features adicionais
def criar_features_adicionais(dados):
    colunas_numeros = ['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']

    # Características estatísticas
    dados['media_numeros'] = dados[colunas_numeros].mean(axis=1)
    dados['std_numeros'] = dados[colunas_numeros].std(axis=1)
    dados['soma_numeros'] = dados[colunas_numeros].sum(axis=1)

    # Diferenças entre números consecutivos
    for i in range(len(colunas_numeros) - 1):
        dados[f'diff_{i+1}'] = dados[colunas_numeros[i+1]] - dados[colunas_numeros[i]]

    # Análise de paridade
    for col in colunas_numeros:
        dados[f'{col}_par'] = dados[col] % 2 == 0
    dados['qtd_pares'] = dados[[f'{col}_par' for col in colunas_numeros]].sum(axis=1)

    return dados

# Função para avaliar a probabilidade de cada sequência
def avaliar_probabilidade(pipeline, sequencia_df):
    sequencia_features = pipeline.transform_input(sequencia_df)
    probabilidade = pipeline.modelo.predict_proba(sequencia_features)
    # Retorna a probabilidade média das previsões
    return np.mean([np.max(probs) for probs in probabilidade])

# Função para simular e gerar sequências com alta probabilidade
def gerar_sequencias_altas_probabilidades(pipeline, num_sequencias=10, num_geracoes=100):
    sequencias_geradas = []
    sequencias_encontradas = []

    for _ in range(num_geracoes):
        sequencia = tuple(sorted(random.sample(range(1, 61), 6)))  # Gera uma nova sequência aleatória
        sequencia_df = pd.DataFrame([sequencia], columns=['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6'])

        # Criar as características adicionais
        sequencia_df = criar_features_adicionais(sequencia_df)

        # Avalia a probabilidade de sucesso da sequência
        probabilidade = avaliar_probabilidade(pipeline, sequencia_df)
        sequencias_geradas.append((sequencia, probabilidade))

    # Seleciona as sequências com maior probabilidade
    sequencias_geradas.sort(key=lambda x: x[1], reverse=True)
    sequencias_encontradas = [seq for seq, prob in sequencias_geradas[:num_sequencias]]

    return sequencias_encontradas

def main():
    # Montar Google Drive
    caminho_base = montar_drive()

    # Carregar o pipeline
    pipeline = carregar_pipeline(caminho_base)
    if pipeline is None:
        print("Erro ao carregar o pipeline. Certifique-se de que ele foi criado anteriormente.")
        return

    # Gerar 10 sequências com alta probabilidade
    sequencias = gerar_sequencias_altas_probabilidades(pipeline, num_sequencias=10, num_geracoes=100)

    # Exibir as previsões
    print("\nPrevisões das 10 sequências com alta probabilidade de acerto:")
    for i, pred in enumerate(sequencias, start=1):
        print(f"Previsão {i}: {pred}")

if __name__ == "__main__":
    main()


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,40 mins 23 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_unknownUser_ss96le
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Carregando o pipeline treinado...

Previsões das 10 sequências com alta probabilidade de acerto:
Previsão 1: (6, 35, 40, 44, 47, 58)
Previsão 2: (8, 10, 30, 34, 45, 57)
Previsão 3: (30, 34, 42, 45, 50, 53)
Previsão 4: (1, 12, 13, 25, 57, 59)
Previsão 5: (6, 10, 11, 33, 39, 58)
Previsão 6: (1, 5, 11, 33, 37, 39)
Previsão 7: (2, 5, 6, 9, 36, 52)
Previsão 8: (18, 37, 42, 44, 52, 53)
Previsão 9: (30, 38, 39, 42, 52, 60)
Previsão 10: (4, 24, 44, 45, 48, 51)


In [31]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from google.colab import drive, files
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import joblib  # Importar joblib para salvar e carregar o modelo

# Função para selecionar o arquivo no Google Colab
def selecionar_arquivo(nome_arquivo):
    """
    Função para selecionar um arquivo .xlsx de sorteios.
    Retorna o caminho do arquivo selecionado.
    """
    print(f"Selecione o {nome_arquivo} (em formato .xlsx):")
    arquivo = files.upload()
    if not arquivo:
        return None
    return next(iter(arquivo))  # Retorna o primeiro arquivo selecionado

# Função para montar o Google Drive
def montar_drive():
    """
    Função para montar o Google Drive no Colab.
    Retorna o caminho do diretório onde o modelo será salvo.
    """
    drive.mount('/content/drive')
    caminho_base = '/content/drive/MyDrive/modelo_loteria/'
    if not os.path.exists(caminho_base):
        os.makedirs(caminho_base)
    return caminho_base

# Função para criar características adicionais
def criar_features_adicionais(dados):
    """
    Cria características adicionais baseadas nas colunas de números de sorteio.
    """
    colunas_numeros = ['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']

    # Características estatísticas
    dados['media_numeros'] = dados[colunas_numeros].mean(axis=1)
    dados['std_numeros'] = dados[colunas_numeros].std(axis=1)
    dados['soma_numeros'] = dados[colunas_numeros].sum(axis=1)

    # Diferenças entre números consecutivos
    for i in range(len(colunas_numeros) - 1):
        dados[f'diff_{i+1}'] = dados[colunas_numeros[i+1]] - dados[colunas_numeros[i]]

    # Análise de paridade
    for col in colunas_numeros:
        dados[f'{col}_par'] = dados[col] % 2 == 0
    dados['qtd_pares'] = dados[[f'{col}_par' for col in colunas_numeros]].sum(axis=1)

    return dados

# Função para aplicar restrições nas sequências
def aplicar_restricoes(dados):
    """
    Aplica restrições para eliminar sequências impossíveis.
    """
    # Exemplo de restrição: sequências com números repetidos são descartadas
    dados = dados[dados[['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']].nunique(axis=1) == 6]
    return dados

# Função para salvar o arquivo no Google Drive
def salvar_arquivo(dados, caminho_base, nome_arquivo):
    """
    Função para salvar o DataFrame 'dados' no Google Drive, sobrescrevendo o arquivo existente.
    """
    caminho_arquivo = os.path.join(caminho_base, nome_arquivo)
    dados.to_excel(caminho_arquivo, index=False, engine='openpyxl')
    print(f"Arquivo salvo com sucesso em: {caminho_arquivo}")

# Função para gerar sequências de previsão com o modelo treinado
def gerar_sequencias_previsao(modelo, X_teste, num_sequencias=10):
    """
    Gera sequências de previsão com base no modelo treinado.
    """
    previsoes = []
    for _ in tqdm(range(num_sequencias), desc="Gerando sequências de previsão", ncols=100):
        # Faz uma previsão usando o modelo
        previsao = modelo.named_steps['classifier'].predict(X_teste)  # Acessando o classificador dentro do pipeline
        previsoes.append(previsao)

    return np.array(previsoes)

# Classe para encapsular o modelo de aprendizado de máquina
class ModeloPipeline:
    def __init__(self, caminho_base):
        self.caminho_base = caminho_base
        self.modelo = self.carregar_modelo()

    def carregar_modelo(self):
        """
        Carrega um modelo salvo previamente ou cria um novo modelo se não houver.
        """
        caminho_modelo = os.path.join(self.caminho_base, 'modelo_sorteio.pkl')
        if os.path.exists(caminho_modelo):
            # Carregar modelo existente
            print("Modelo encontrado e carregado!")
            return joblib.load(caminho_modelo)
        else:
            # Criar um novo modelo
            print("Criando um novo modelo!")
            return Pipeline([
                ('imputer', SimpleImputer(strategy='mean')),  # Imputação de valores ausentes
                ('scaler', StandardScaler()),  # Normalização dos dados
                ('classifier', RandomForestClassifier())  # Classificador RandomForest
            ])

    def incremental_fit(self, X_treino, y_treino):
        """
        Incrementa o aprendizado do modelo com os dados fornecidos.
        """
        self.modelo.fit(X_treino, y_treino)
        print("Modelo treinado ou incrementado com novos dados.")

    def save_model(self):
        """
        Salva o modelo treinado no Google Drive.
        """
        caminho_modelo = os.path.join(self.caminho_base, 'modelo_sorteio.pkl')
        joblib.dump(self.modelo, caminho_modelo)
        print(f"Modelo salvo em: {caminho_modelo}")

# Função principal para processar os dados, treinar o modelo e gerar previsões
def processar_e_treinar():
    """
    Função principal que carrega os dados, aplica as transformações, treina o modelo e gera previsões.
    """
    caminho_base = montar_drive()

    # Carregar dados
    caminho_arquivo_crescente = selecionar_arquivo("arquivo com dados em ordem crescente")
    if not caminho_arquivo_crescente:
        print("Nenhum arquivo foi selecionado.")
        return

    dados_crescentes = pd.read_excel(caminho_arquivo_crescente, engine='openpyxl')
    print("Colunas do arquivo em ordem crescente carregadas:", dados_crescentes.columns)

    # Criar características adicionais
    dados_crescentes = criar_features_adicionais(dados_crescentes)

    # Aplicar restrições nas sequências
    dados_crescentes = aplicar_restricoes(dados_crescentes)

    # Salvar o arquivo processado
    salvar_arquivo(dados_crescentes, caminho_base, 'Base_dados_processada.xlsx')

    # Preparar os dados para o treinamento
    X = dados_crescentes[['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6', 'media_numeros', 'std_numeros', 'qtd_pares']]
    y = dados_crescentes[['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']]

    # Criar o modelo
    modelo = ModeloPipeline(caminho_base)

    # Incrementar o modelo com os novos dados
    modelo.incremental_fit(X, y)

    # Salvar o modelo treinado
    modelo.save_model()

    # Gerar previsões após o treinamento
    sequencias_previstas = gerar_sequencias_previsao(modelo, X, num_sequencias=10)
    print("Sequências geradas: ", sequencias_previstas)

if __name__ == "__main__":
    processar_e_treinar()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Selecione o arquivo com dados em ordem crescente (em formato .xlsx):


Saving sorteio.xlsx to sorteio (11).xlsx
Colunas do arquivo em ordem crescente carregadas: Index(['Sorteio', 'Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5',
       'Numero6'],
      dtype='object')
Modelo encontrado e carregado!
Modelos salvos com sucesso!
Previsões geradas: [[[ 4  5 30 33 41 52]
  [ 9 37 39 41 43 49]
  [10 11 29 30 36 47]
  ...
  [ 2  3 11 25 37 43]
  [16 22 33 34 49 59]
  [ 7  9 25 37 57 59]]

 [[ 4  5 30 33 41 52]
  [ 9 37 39 41 43 49]
  [10 11 29 30 36 47]
  ...
  [ 2  3 11 25 37 43]
  [16 22 33 34 49 59]
  [ 7  9 25 37 57 59]]

 [[ 4  5 30 33 41 52]
  [ 9 37 39 41 43 49]
  [10 11 29 30 36 47]
  ...
  [ 2  3 11 25 37 43]
  [16 22 33 34 49 59]
  [ 7  9 25 37 57 59]]]
Não foi possível localizar o VotingClassifier.
